Bike Sharing Demand
1. EDA 
    - Data Summary
    - Visualization 
    - Correlation
    - Feature Engineering
2. Modeling

**EDA**

1. Data Summary

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

1. season - (1:봄, 2: 여름, 3: 가을, 4:겨울)
2. hoilday - 휴일 고려
3. workingday - 휴일 주말 상관없이 일하는 날
4. weather - (1: 맑음, 2:안개, 3:가벼운 눈,비, 4:무거운 눈,비)
5. temp - 온도
6. atemp  - 날씨 (체감 지수)
7. windspeed- 바람 세기
8. casual-등록되지 않은 사용자 임대가 시작된 횟수
9. register-등록된 사용자 대여 횟수로 시작됨
10. Count-총 임대 횟수(종속 변수)

In [2]:
train_csv = pd.read_csv('../input/train.csv')
test_csv = pd.read_csv('../input/test.csv')
sub = pd.read_csv('../input/sampleSubmission.csv')

In [3]:
train_csv.head(5)

1. datetime = date와 Hour로 구분할 수 있다. 
2. date = Month와 weekday로 구분을 할 수 있다.

In [4]:
# datetime 변수를 여러개로 나누어 보자.
train_csv['date']  = train_csv.datetime.apply(lambda x: x.split()[0])
train_csv['hour'] = train_csv.datetime.apply(lambda x: x.split()[1].split(':')[0])
train_csv['weekday'] = train_csv.date.apply(lambda dateString : datetime.strptime(dateString, '%Y-%m-%d').weekday())
train_csv['month'] = train_csv.date.apply(lambda dateString : datetime.strptime(dateString, '%Y-%m-%d').month)

In [5]:
train_csv.describe()

Describe를 보면 종속변수(count)가 Min(최솟값)과 Max(최댓값)의 차이가 크다는 것을 알 수 있다.
Count를 살펴보도록 하자.

In [6]:
%matplotlib inline
fig, axes = plt.subplots(figsize=(15,10),ncols=2,nrows= 1)
sns.distplot(train_csv['count'],ax=axes[0])
sns.distplot(train_csv['count'].apply(lambda x: np.log1p(x)),ax=axes[1])
plt.show()

위의 그래프는 Count의 분포도를 보여준다. 왼쪽이 로그처리하기 전의 그래프로써
0-200사이에 매우 많이 밀집한 것을 볼 수 있다. 즉, Skewed된 멱함수 분포를 띄고 있다고 
볼 수 있기 때문에 오른쪽 그래프처럼 정규분포를 따르게 해서 볼 수 있다.

Count와 다른 variable과의 관계를 살펴보도록하자.

In [7]:
sns.boxplot(data = train_csv,y='count',orient='v')
plt.show()

위의 그래프를 보면 count의 특이치(*)가 매우 많은 것을 알 수 있다.
즉, Outlier가 많다고 볼 수 있는데 이것들이 얼마나 존재를하는지 다른 변수
에게서도 찾아보자
우선적으로 날씨에 대한 컬럼과 Categorical 컬럼 과 나누어 보도록하자.

In [8]:
fig,axes = plt.subplots(ncols=2 ,nrows=2)
fig.set_size_inches(15,10)
sns.boxplot(data=train_csv,x='season',y='count',ax=axes[0][0])
sns.boxplot(data=train_csv,x='holiday',y='count',ax=axes[0][1])
sns.boxplot(data=train_csv,x='workingday',y='count',ax=axes[1][0])
sns.boxplot(data=train_csv,x='weather',y='count',ax=axes[1][1])

fig1,axes1 = plt.subplots()
fig1.set_size_inches(15,10)
sns.boxplot(data=train_csv,x='hour',y='count')

위의 그래프를 보면 몇가지를 알 수 있다, 
1. Season은 봄이 여름 ,가을,겨울에 비해서 Count가 낮다는 것을 알 수 있다.
2. Count에 대한 Outlier가 꽤 많다는 것을 알 수 있다.
3, Workingday를 보면 일을 하지 않을 때 보다 일을 할 경우 outlier가 높다.(일을 할 경우 더많은 이용을 한다.)
4. hour를 보면 08AM과 17PM에 가장 많은 분포를 보이는 것을 알 수 있다.

그렇다면 날씨(temp,atemp,humidity,windspeed)와 Count의 관계를
살펴보도록하자.
날씨의 데이터는 Numeric 데이터로 볼 수 있다.
그렇기에 시각화에 앞서 상관계수를 구하고 시각화를 진행하도록 하자.

In [9]:
corrs = train_csv[['temp','atemp','windspeed','humidity','count']].corr()
sns.heatmap(corrs,annot=True,vmax=3)

In [ ]:
fig,axes = plt.subplots(ncols=2,nrows=2)
fig.set_size_inches(15,10)
sns.regplot(data=train_csv,x='temp',y='count',ax=axes[0][0])
sns.regplot(data=train_csv,x='atemp',y='count',ax=axes[0][1])
sns.regplot(data=train_csv,x='humidity',y='count',ax=axes[1][0])
sns.regplot(data=train_csv,x='windspeed',y='count',ax=axes[1][1])
plt.show()

**위의 그래프를 보면 날씨와의 관계도 어느정도 파악할 수 있다.


 Count와 다른 variable을 파악을 해보았다. 
 다음으로 Count의 Outlier를 삭제하고 Count를 살펴보고난 후 상관분석을 통해 필요한 컬럼을 파악하자. 이후 모델링을 진해해서 결과를 보도록하자.

In [ ]:
#Remove Outlier
train_csv_Outliers = train_csv[np.abs(train_csv["count"]-train_csv["count"].mean())<=(2*train_csv["count"].std())]
print("Oulier포함:",train_csv.shape)
print("Outlier제거:",train_csv_Outliers.shape)

In [ ]:
fig,axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(15,10)
sns.distplot(train_csv['count'],ax=axes[0][0])
sns.distplot(np.log1p(train_csv['count']),ax=axes[0][1])
sns.distplot(train_csv_Outliers['count'],ax=axes[1][0])
sns.distplot(np.log1p(train_csv['count']),ax=axes[1][1])
plt.show()

In [ ]:
fig,axes = plt.subplots()
fig.set_size_inches(15,10)
corrs = train_csv[['season', 'holiday', 'workingday', 'weather','date',
       'hour', 'weekday', 'month','temp','atemp','windspeed','humidity','registered','casual','count']].corr()
sns.heatmap(corrs,annot=True)

In [ ]:
corrs.iloc[:,-1]

**Modeling**

In [ ]:
Train = pd.read_csv('../input/train.csv')
Test = pd.read_csv('../input/test.csv')

In [ ]:
# datetime 변수를 여러개로 나누어 보자.
Train['date']  = Train.datetime.apply(lambda x: x.split()[0])
Train['hour'] = Train.datetime.apply(lambda x: x.split()[1].split(':')[0])
Train['weekday'] = Train.date.apply(lambda dateString : datetime.strptime(dateString, '%Y-%m-%d').weekday())
Train['month'] = Train.date.apply(lambda dateString : datetime.strptime(dateString, '%Y-%m-%d').month)

In [ ]:
# datetime 변수를 여러개로 나누어 보자.
Test['date']  = Test.datetime.apply(lambda x: x.split()[0])
Test['hour'] = Test.datetime.apply(lambda x: x.split()[1].split(':')[0])
Test['weekday'] = Test.date.apply(lambda dateString : datetime.strptime(dateString, '%Y-%m-%d').weekday())
Test['month'] = Test.date.apply(lambda dateString : datetime.strptime(dateString, '%Y-%m-%d').month)

In [ ]:
season_dm = pd.get_dummies(Train['season'],prefix='season')
weather_dm = pd.get_dummies(Train['weather'],prefix='weather')
hour_dm = pd.get_dummies(Train['hour'],prefix='hour')
weekday_dm = pd.get_dummies(Train['wwekday'],prefix = 'weekday')
month_dm = pd.get_dummies(Train['month'],prefix='month')
season_dm1 = pd.get_dummies(Test['season'],prefix='season')
weather_dm1 = pd.get_dummies(Test['weather'],prefix='weather')
hour_dm1 = pd.get_dummies(Test['hour'],prefix='hour')
weekday_dm1 = pd.get_dummies(Test['wwekday'],prefix = 'weekday')
month_dm1 = pd.get_dummies(Test['month'],prefix='month')

InputX = InputX.join(season_dm)
InputX = InputX.join(weather_dm)
InputX = InputX.join(hour_dm)
InputX = InputX.join(weekday_dm)
InputX = InputX.join(month_dm)
TestX = TestX.join(season_dm1)
TestX = TestX.join(weather_dm1)
TestX = TestX.join(hour_dm1)
TestX = TestX.join(weekday_dm1)
TestX = TestX.join(month_dm1)

In [ ]:
print(TestX.shape)
print(InputX.)